## Balance Sheet

In [182]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import string
pd.options.display.float_format = '{:.0f}'.format
import requests


symbol = 'MSFT'

url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol

# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above
page = requests.get(url, headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element
tree.xpath("//h1/text()")

['Microsoft Corporation (MSFT)']

In [183]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

# Ensure that some table rows are found; if none are found, then it's possible
# that Yahoo Finance has changed their page layout, or have detected
# that you're scraping the page.
assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

,0,1,2,3,4
0,Breakdown,6/30/2020,6/30/2019,6/30/2018,6/30/2017
1,Total Assets,"301,311,000","286,556,000","258,848,000","241,086,000"
2,Total Liabilities Net Minority Interest,"183,007,000","184,226,000","176,130,000","168,692,000"
3,Total Equity Gross Minority Interest,"118,304,000","102,330,000","82,718,000","72,394,000"
4,Total Capitalization,"177,882,000","168,992,000","154,960,000","148,467,000"
5,Common Stock Equity,"118,304,000","102,330,000","82,718,000","72,394,000"
6,Capital Lease Obligations,"7,671,000","6,188,000","5,568,000",NaN
7,Net Tangible Assets,"67,915,000","52,554,000","38,982,000","27,166,000"
8,Working Capital,"109,605,000","106,132,000","111,174,000","95,324,000"
9,Invested Capital,"181,631,000","174,508,000","158,958,000","158,588,000"


In [189]:
df = pd.DataFrame(parsed_rows)
df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

# Rename the "Breakdown" column to "Date"
cols = list(df.columns)
cols[0] = 'Date'
df = df.set_axis(cols, axis='columns', inplace=False)

df.T

,1,2,3,4
Date,6/30/2020,6/30/2019,6/30/2018,6/30/2017
Total Assets,"301,311,000","286,556,000","258,848,000","241,086,000"
Total Liabilities Net Minority Interest,"183,007,000","184,226,000","176,130,000","168,692,000"
Total Equity Gross Minority Interest,"118,304,000","102,330,000","82,718,000","72,394,000"
Total Capitalization,"177,882,000","168,992,000","154,960,000","148,467,000"
Common Stock Equity,"118,304,000","102,330,000","82,718,000","72,394,000"
Capital Lease Obligations,"7,671,000","6,188,000","5,568,000",NaN
Net Tangible Assets,"67,915,000","52,554,000","38,982,000","27,166,000"
Working Capital,"109,605,000","106,132,000","111,174,000","95,324,000"
Invested Capital,"181,631,000","174,508,000","158,958,000","158,588,000"


In [185]:
df.dtypes

Date                                       object
Total Assets                               object
Total Liabilities Net Minority Interest    object
Total Equity Gross Minority Interest       object
Total Capitalization                       object
Common Stock Equity                        object
Capital Lease Obligations                  object
Net Tangible Assets                        object
Working Capital                            object
Invested Capital                           object
Tangible Book Value                        object
Total Debt                                 object
Net Debt                                   object
Share Issued                               object
Ordinary Shares Number                     object
dtype: object

In [186]:
numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

for column_name in numeric_columns:
    df[column_name] = df[column_name].str.replace(',', '') # Remove the thousands separator
    df[column_name] = df[column_name].astype(np.float64) # Convert the column to float64

df.dtypes

Date                                        object
Total Assets                               float64
Total Liabilities Net Minority Interest    float64
Total Equity Gross Minority Interest       float64
Total Capitalization                       float64
Common Stock Equity                        float64
Capital Lease Obligations                  float64
Net Tangible Assets                        float64
Working Capital                            float64
Invested Capital                           float64
Tangible Book Value                        float64
Total Debt                                 float64
Net Debt                                   float64
Share Issued                               float64
Ordinary Shares Number                     float64
dtype: object

In [187]:
df

,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,Working Capital,Invested Capital,Tangible Book Value,Total Debt,Net Debt,Share Issued,Ordinary Shares Number
1,6/30/2020,301311000.00%,183007000.00%,118304000.00%,177882000.00%,118304000.00%,7671000.00%,67915000.00%,109605000.00%,181631000.00%,67915000.00%,70998000.00%,49751000.00%,7571000.00%,7571000.00%
2,6/30/2019,286556000.00%,184226000.00%,102330000.00%,168992000.00%,102330000.00%,6188000.00%,52554000.00%,106132000.00%,174508000.00%,52554000.00%,78366000.00%,60822000.00%,7643000.00%,7643000.00%
3,6/30/2018,258848000.00%,176130000.00%,82718000.00%,154960000.00%,82718000.00%,5568000.00%,38982000.00%,111174000.00%,158958000.00%,38982000.00%,81808000.00%,64294000.00%,7677000.00%,7677000.00%
4,6/30/2017,241086000.00%,168692000.00%,72394000.00%,148467000.00%,72394000.00%,nan%,27166000.00%,95324000.00%,158588000.00%,27166000.00%,86194000.00%,78531000.00%,7708000.00%,7708000.00%


## Cash Flow Statement

In [177]:
api_key = 'ae404d758dcff353b68cf1af6ac9621c'
symbol = 'MSFT'

CF = requests.get(f'https://fmpcloud.io/api/v3/cash-flow-statement/{symbol}?apikey={api_key}').json()

count = 0
#Create an empty dictionary 
CF_3Y = {}


In [178]:
for item in CF:
  if count < 3:
    date = item['date']
    CF_3Y[date] = item
    count += 1


In [179]:
CF_Common_Size = pd.DataFrame.from_dict(CF_3Y, orient='index')
CF_Common_Size = CF_Common_Size.T
CF_Common_Size

,2020-06-30,2019-06-30,2018-06-30
date,2020-06-30,2019-06-30,2018-06-30
symbol,MSFT,MSFT,MSFT
fillingDate,2020-07-30,2019-08-01,2018-08-03
acceptedDate,2020-07-30 20:44:46,2019-08-01 16:09:55,2018-08-03 11:03:33
period,FY,FY,FY
netIncome,44281000000,39240000000,16571000000
depreciationAndAmortization,12796000000,11682000000,10261000000
deferredIncomeTax,-3620000000,-3534000000,13040000000
stockBasedCompensation,5289000000,4652000000,3940000000
changeInWorkingCapital,2148000000,937000000,2284000000


## Income Statement

In [180]:

IS = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{symbol}?apikey={api_key}').json()


Income_statement = pd.DataFrame()
Income_statement = output.append(IS, ignore_index=True)
print(Income_statement.head())

         date symbol fillingDate         acceptedDate period       revenue  \
0  2020-06-30   MSFT  2020-07-30  2020-07-30 20:44:46     FY  143015000000   
1  2019-06-30   MSFT  2019-08-01  2019-08-01 16:09:55     FY  125843000000   
2  2018-06-30   MSFT  2018-08-03  2018-08-03 11:03:33     FY  110360000000   
3  2017-06-30   MSFT  2017-08-02  2017-08-02 16:15:01     FY   89950000000   
4  2016-06-30   MSFT  2016-07-28  2016-07-28 16:12:18     FY   85320000000   

   costOfRevenue  grossProfit  grossProfitRatio  \
0    46078000000  96937000000             0.68%   
1    42910000000  82933000000             0.66%   
2    38353000000  72007000000             0.65%   
3    34261000000  55689000000             0.62%   
4    32780000000  52540000000             0.62%   

   researchAndDevelopmentExpenses  ...  incomeBeforeTaxRatio  \
0                     19269000000  ...                 0.37%   
1                     16876000000  ...                 0.35%   
2                     1472600000

In [188]:
Income_statement.T

,0,1,2,3,4
date,2020-06-30,2019-06-30,2018-06-30,2017-06-30,2016-06-30
symbol,MSFT,MSFT,MSFT,MSFT,MSFT
fillingDate,2020-07-30,2019-08-01,2018-08-03,2017-08-02,2016-07-28
acceptedDate,2020-07-30 20:44:46,2019-08-01 16:09:55,2018-08-03 11:03:33,2017-08-02 16:15:01,2016-07-28 16:12:18
period,FY,FY,FY,FY,FY
revenue,143015000000,125843000000,110360000000,89950000000,85320000000
costOfRevenue,46078000000,42910000000,38353000000,34261000000,32780000000
grossProfit,96937000000,82933000000,72007000000,55689000000,52540000000
grossProfitRatio,0.68%,0.66%,0.65%,0.62%,0.62%
researchAndDevelopmentExpenses,19269000000,16876000000,14726000000,13037000000,11988000000
